In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip "/content/drive/My Drive/Visdrone Dataset/VisDrone2019-DET-train.zip" -d /content/visdrone

Streaming output truncated to the last 5000 lines.
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000140.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000141.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000142.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000143.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000144.jpg  
  inflating: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000145.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000146.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000147.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000148.jpg  
 extracting: /content/visdrone/VisDrone2019-DET-train/images/9999937_00000_d_0000149.jpg  
  inflating: /content/visdrone/VisDrone

In [4]:
!unzip "/content/drive/My Drive/Visdrone Dataset/VisDrone2019-DET-val.zip" -d /content/visdrone

Archive:  /content/drive/My Drive/Visdrone Dataset/VisDrone2019-DET-val.zip
   creating: /content/visdrone/VisDrone2019-DET-val/
  inflating: /content/visdrone/VisDrone2019-DET-val/.DS_Store  
   creating: /content/visdrone/VisDrone2019-DET-val/annotations/
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_02999_d_0000005.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_03499_d_0000006.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_03999_d_0000007.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_04527_d_0000008.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_05249_d_0000009.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_05499_d_0000010.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_05999_d_0000011.txt  
  inflating: /content/visdrone/VisDrone2019-DET-val/annotations/0000001_07999_d_0000

In [5]:
# After unzipping, update paths to include subfolders:
train_dir = '/content/visdrone/VisDrone2019-DET-train'
val_dir = '/content/visdrone/VisDrone2019-DET-val'

In [6]:
import os
import cv2
from PIL import Image
import pandas as pd

def visdrone_to_yolo(img_dir,ann_dir,yolo_ann_dir):
    os.makedirs(yolo_ann_dir, exist_ok=True)
    # loop that goes through annotation directory
    for ann_file in os.listdir(ann_dir):
        if not ann_file.endswith('.txt'):
            continue

        img_file = ann_file.replace('.txt','.jpg')
        img_path = os.path.join(img_dir, img_file)
        ann_path = os.path.join(ann_dir, ann_file)
        yolo_ann_path = os.path.join(yolo_ann_dir, ann_file)

        if not os.path.exists(img_path):
            print(f"Image does not exist for annotation: {ann_file}, skipping.")
            continue


        img = Image.open(img_path)
        w, h = img.size

        yolo_ann_lines = []
        with open(ann_path,'r')as file:
            for lines in file:
                parts = lines.strip().split(',')
                if len(parts) < 8:
                    continue
                x, y, width, height, score, category, trunc, occ = map(float, parts[:8])
                category = int(category)

                # skip ignored regions
                if category == 0:
                    continue

                yolo_cls = category - 1
                x_center =  (x + width/2 ) / w
                y_center =  (y + height/2 ) / h
                norm_width = width / w
                norm_height = height / h

                yolo_ann_lines.append(f"{yolo_cls} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

        if yolo_ann_lines:
            with open(yolo_ann_path, 'w') as file:
                file.write('\n'.join(yolo_ann_lines))
        else:
            print(f"Skipping empty label file: {ann_file}")



visdrone_to_yolo(
    ann_dir=os.path.join(train_dir, 'annotations'),
    img_dir=os.path.join(train_dir, 'images'),
    yolo_ann_dir= os.path.join(train_dir,'labels')
)

visdrone_to_yolo(
    ann_dir=os.path.join(val_dir, 'annotations'),
    img_dir=os.path.join(val_dir, 'images'),
    yolo_ann_dir= os.path.join(val_dir,'labels')
)


In [7]:
import os

os.makedirs('/content/dataset', exist_ok=True)

yaml_content = """
path: /content/visdrone
train: VisDrone2019-DET-train/images
val: VisDrone2019-DET-val/images

names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
"""

with open('/content/dataset/visdrone.yaml', 'w') as f:
    f.write(yaml_content)

In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [9]:
import os

def clean_label_dir(labels_dir, max_class_idx=9):     #Removes lines from label files where the class index exceeds max_class_idx.


     for fname in os.listdir(labels_dir):
        if not fname.endswith('.txt'):
            continue  # skip non-label files
        fpath = os.path.join(labels_dir, fname)
        with open(fpath, 'r') as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            try:
                class_idx = int(line.split()[0])
                if class_idx <= max_class_idx:
                    new_lines.append(line)
            except Exception as e:
                print(f"Skipping line in {fname}: {line.strip()} (Error: {e})")
        with open(fpath, 'w') as f:
            f.writelines(new_lines)

# Example usage:
train_labels_dir = '/content/visdrone/VisDrone2019-DET-train/labels'
val_labels_dir = '/content/visdrone/VisDrone2019-DET-val/labels'
clean_label_dir(train_labels_dir, max_class_idx=9)
clean_label_dir(val_labels_dir, max_class_idx=9)

In [10]:
from ultralytics import YOLO
from google.colab import drive

# Mount Google Drive to access saved checkpoints
drive.mount('/content/drive')

# Load the last saved checkpoint
model = YOLO('/content/drive/MyDrive/visdrone_train/yolov82/weights/last.pt')

# Resume training from where it left off
model.train(resume=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/visdrone.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, 

100%|██████████| 755k/755k [00:00<00:00, 24.7MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 5.35M/5.35M [00:00<00:00, 118MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.2 ms, read: 82.4±31.5 MB/s, size: 261.9 KB)


...

                   all        548      38759      0.528      0.408      0.427      0.259



43 epochs completed in 3.305 hours.
Optimizer stripped from /content/drive/MyDrive/visdrone_train/yolov82/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/visdrone_train/yolov82/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/visdrone_train/yolov82/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:13<00:00,  2.66it/s]


                   all        548      38759       0.53      0.407      0.427       0.26
            pedestrian        520       8844      0.555      0.436      0.476      0.225
                people        482       5125       0.59      0.329       0.39      0.154
               bicycle        364       1287      0.318      0.184       0.16     0.0762
                   car        515      14064      0.735        0.8      0.828      0.594
                   van        421       1975       0.54      0.462      0.476      0.339
                 truck        266        750      0.509      0.383      0.385      0.262
              tricycle        337       1045      0.456      0.296      0.303      0.174
       awning-tricycle        220        532      0.328      0.211      0.181      0.118
                   bus        131        251      0.696      0.514       0.58      0.435
                 motor        485       4886      0.571      0.458      0.487      0.221
Speed: 0.4ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2ff4be09d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [12]:
#Evaluating the trained yolov8 model
from ultralytics import YOLO

# Load the trained model
model = YOLO("/content/drive/MyDrive/visdrone_train/yolov82/weights/last.pt")  # replace with your trained model path

# Evaluate the model on the validation set
metrics = model.val(data="/content/dataset/visdrone.yaml", split='val')

# Print key metrics
print("mAP50-95:", metrics.box.map)       # mean Average Precision (mAP) @ IoU=0.5:0.95
print("mAP50:", metrics.box.map50)        # mAP @ IoU=0.5
print("Precision:", metrics.box.mp)
print("Recall:", metrics.box.mr)
print("F1 Score:", metrics.box.f1)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2058.6±949.2 MB/s, size: 153.1 KB)


val: Scanning /content/visdrone/VisDrone2019-DET-val/labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:13<00:00,  2.62it/s]


                   all        548      38759      0.528      0.409      0.428      0.261
            pedestrian        520       8844      0.554      0.438      0.477      0.226
                people        482       5125      0.591      0.332      0.393      0.156
               bicycle        364       1287      0.315      0.184       0.16      0.077
                   car        515      14064      0.735      0.802      0.828      0.596
                   van        421       1975      0.536      0.465      0.476       0.34
                 truck        266        750      0.507      0.383      0.386      0.262
              tricycle        337       1045      0.457      0.298      0.303      0.175
       awning-tricycle        220        532      0.324      0.211      0.181      0.117
                   bus        131        251      0.693      0.518      0.584      0.436
                 motor        485       4886       0.57      0.461      0.487      0.222
Speed: 2.1ms preproce